In [ ]:
import numpy as np
import dimers as dim
import CleanKagomeFunctions as lattice

In [ ]:
def createdualtable(L):
    '''
        Creates the table of dual bonds corresponding to the dual lattice 
        of side size L.
        Returns a table identifing an int with the three coordinates of 
        the dual bond and a dictionnary identifying the
        three coordinates with the dual bond's int index. This allows to 
        handle other relations between dual bonds in an easier way.
        > d_ijl: table dimer -> coordinates
        > ijl_d: dictionnary coordinates -> dimer
    '''
    return lattice.createdualtable(L)

In [ ]:
def createspinsitetable(L):
    '''
        Creates the table of spin sites corresponding to a real space 
        lattice with dual of site size L.
        Returns a table identifing an int with the coordinates of the 
        spin site and a dictionnary identifying the
        three coordinates with the spin site's int index. This allows 
        to handle other relations between spin sites in an easier way.
        > d_ijl: table spin -> coordinates
        > ijl_d: dictionnary coordinates -> spins
    '''
    return lattice.creatspinsitetable(L)

In [ ]:
def dualbondspinsitelinks(d_ijl, ijl_s, L):
    '''
        For a lattice with side size L, this function  returns two tables:
        > d_2s: for each dual bond, which are the 2spin sites around it.
        > s2_d: for each pair of spin sites nearest to one another, which 
        is the dual bond between them (dictionary)
    '''
    return lattice.dualbondspinsitelinks(d_ijl, ijl_s, L)

In [ ]:
def spins_dimers_for_update(s_ijl, ijl_s, s2_d, L):
    '''
        Returns a list of spin site indices and a list of dual bond indices. 
        Going through the spins list allows to map the whole
        spin state of the system. The ith dimer lies between the ith and ith+1 
        spin.
        > spinsiteslist: list of spin sites in the update order
        > dualbondslist: list of dual bonds in the update order
    '''
    return lattice.spins_dimers_for_update(s_ijl, ijl_s, s2_d, L)

In [ ]:
def nsitesconnections(d_ijl, ijl_d, L):
    '''
        For each dual bond, which are the other dual bonds which are 
        touching it through an "n" site (in the kagomé case, that's a 
        site with 6 dualbonds)
        > d_nd: array[d] = list of dual bonds connected to d by a n-site
    '''
    return lattice.nsitesconnections(d_ijl, ijl_d, L)

In [ ]:
def vsitesconnections(d_ijl, ijl_d, L):
    '''
        For each dual bond, which are the other dual bonds which are 
        touching it through an "v" site (in the kagomé case, that's a 
        site with 3 dual bonds)
        > d_vd: array[d] = list of dual bonds connected to d by a v-site
    '''
    return lattice.vsitesconnections(d_ijl, ijl_d, L)

In [ ]:
def windingtable(d_ijl, L):
    '''
        For each dual bond, is it on one of the two lines which are used 
        to count the winding numbers?
        > d_wn: array[d] = [w1? w2?]
    '''
    return lattice.windingtable(d_ijl, L)

In [ ]:
def Hamiltonian(couplings, d_ijl, ijl_d, L):
    '''
        Hamitlonian returns a list of couples (coupling value, interaction 
        table) where the interaction table states for each dual bond which 
        are the dual bonds intereacting with it with the mentionned coupling.
        < couplings : dict. for each coupling ('J1', 'J2', 'J3', 'J3st', ...), associates the
        value (use like a matlab struct)
        > hamiltonian = [J1, (J2, d_J2d), (J3, d_J3d), ...]
    '''
    hamiltonian = [J1]
    if 'J2' in couplings: # checks if key in dict.keys() but more efficiently (hash)
        J2 = couplings('J2');
        d_J2d = lattice.d_J2d(d_ijl, ijl_d, L)
        hamiltonian.append((J2, d_J2d))

    if 'J3' in couplings:
        J3 = couplings('J3');
        d_J3d = lattice.d_J3d(d_ijl, ijl_d, L)
        hamiltonian.append((J3,d_J3d))
            
    if 'J3st' in couplings:
        J3st = couplings('J3st');
        J3st = J3st/2.0 # we are going to write two paths going each way!
        d_J3std = lattice.d_J3std(d_ijl, ijl_d, L)
        hamiltonian.append((J3st,d_J3std))
    
    if 'J4' in couplings:
        J4 = couplings('J4');
        d_J4d = lattice.d_J4d(d_ijl, ijl_d, L)
        hamiltonian.append((J4,d_J4d))
        
    return hamiltonian

In [ ]:
def compute_energy(hamiltonian, state, latsize = 1):
    '''
        Computes the energy of the state state given the hamiltonian 
        (from dualwormfunctions.Hamiltonian) and the lattice size 
        (number of sites)
    '''
    return dim.hamiltonian(hamiltonian, state)/latsize

In [ ]:
############### Neighbour pairs #####################

In [ ]:
def reducedgraph(L, s_ijl, ijl_s):
    '''
        Returns exactly one position per spin coordinate.
    '''
    return lattice.reducedgraph(L, s_ijl, ijl_s)

In [ ]:
def sitepairslist(srefs, s_pos, n1, n2, Leff, distmax):
    '''
        For a given structure, this function returns a table containing,
        for each pair (coord s1, coord s2) at distance less than Leff/2, 
        the corresponding distance R and the *indices* s1 and s2 of the 
        spins in positions these positions. 
        We only consider couples containing spins in srefs.
        It returns as well an ordered list of the distances
        and a dictionary associating each distance to a set of spins.
    '''
    
     # for each distance, we get the various spins that are at this distance from a given spin index

    pairs = []
    distmin = Leff
    
   
    for s1 in srefs:
        for s2 in range(len(s_pos)):
            (consider, dist) = lattice.pairseparation(s1, s2, s_pos, n1, n2, Leff, distmax)
            if consider:
                if dist < distmin:
                    distmin = dist
                
                pairs.append(((s1, s2), dist))
                
    distances = []
    distances_spins = {}
    for (spair, dist) in pairs:
        dist = np.round(dist, 4)
        if dist != 0:
            if dist in distances:
                distances_spins[dist].append(spair)
            else:
                distances.append(dist)
                distances_spins[dist] = [spair]

    return pairs, sorted(distances), distances_spins

In [ ]:
def dist_sitepairs(s_pos,  n1, n2, Leff):
    '''
        Using sitepairslist, this function returns a list of (sorted) distances and 
        a dictionnary associating each distance with a spin pair.
        < s_pos: spins-> position
        < n1, n2, Leff: output of superlattice, that is, the structure of the PBC.
        > sorted(distances), distances_spins
    '''
    pairs = lattice.sitepairslist(s_pos, n1, n2, Leff)
    distances = []
    distances_spins = {}
    for (spair, spospair, dist) in pairs:
        dist = np.round(dist, 4)
        if dist in distances:
            distances_spins[dist].append(spair)
        else:
            distances.append(dist)
            distances_spins[dist] = [spair]

    return sorted(distances), distances_spins

In [ ]:
def NeighboursList(L, distmax)
    '''
        Returns a list of distances between sites (smaller than distmax)
        with respect to the lattice reference sites (e.g. 3 for kagome),
        a dictionary of pairs of sites at a given distance and a list of
        the neighbours associated with a given site and distance.
    '''
    #dimer table and dictionary:
    (d_ijl, ijl_d) = lattice.createdualtable(L)
    #spin table and dictionary
    (s_ijl, ijl_s) = lattice.createspinsitetable(L)
    #two spins surrounding each dimer
    (d_2s, s2_d) = lattice.dualbondspinsitelinks(d_ijl, ijl_s, L)
    #dimer-dimer connection through entry sites
    d_nd = lattice.nsitesconnections(d_ijl, ijl_d)
    #dimer-dimer connection through vertex sites
    d_vd = lattice.vsitesconnections(d_ijl, ijl_d, L)
    #for each dimer, is it takeing into account in winding number 1 or 2?
    d_wn = lattice.windingtable(d_ijl, L)
    #list of spin indices and dimer indices for the loop allowing to update the spin state
    (sidlist, didlist) = lattice.spins_dimers_for_update(s_ijl, ijl_s, s2_d, L)
    
    (s_pos, ijl_pos) = lattice.reducedgraph(L, s_ijl, ijl_s)
    pos = list(s_pos.values())
    pos = [list(np.round(posval, 4)) for posval in pos]
    
    #initialise the superlattice
    (n1, n2, Leff, S) = lattice.superlattice(L)
    
    # getting the list of pairs that we're interested in, 
    srefs = lattice.referenceSpins(L, ijl_s)
    pairs, distances, distances_spins = sitepairslist(srefs, s_pos, n1,n2,Leff,distmax)
    
    NNList = [[[] for i in range(len(distances))] for j in range(len(srefs))]
    
    for i in range(len(distances)):
        for pair in distances_spins[distances[i]]:
            for j in range(len(srefs)):
                if srefs[j] in pair:
                    NNList[j][i].append(pair)


    NNList = lattice.inequivalentSites()                
    return distances, distances_spins, NNList, s_pos, srefs
    

In [ ]:
############## STATES INIT ##############################3

In [ ]:
def create_temperatures(nt_list, t_list):
    assert(len(t_list) == len(nt_list) + 1)
    nt = 0
    for nte in nt_list:
        nt += nte

    temp_states = np.zeros(nt)

    nt_start = 0
    for id_nt, nte in enumerate(nt_list):
        temp_states[nt_start: nte+nt_start] = np.linspace(t_list[id_nt], t_list[id_nt + 1], nte)
        nt_start += nte

    return np.unique(temp_states)

In [ ]:
def create_log_temperatures(nt_list, t_list):
    assert(len(t_list) == len(nt_list) + 1)
    nt = 0
    for nte in nt_list:
        nt += nte

    temp_states = np.zeros(nt)

    ## Here, we have htat nt_list = [number between t0 and t1, number between t1 and t2, ...]

    nt_start = 0
    for id_nt, nte in enumerate(nt_list):
        temp_states[nt_start: nt_start + nte] = np.logspace(np.log10(t_list[id_nt]), np.log10(t_list[id_nt +1 ]), nte, endpoint=True)
        print(nt_start, nt_start+nte)
        nt_start +=nte
        
    return np.unique(temp_states)

In [ ]:
def statesinit(nt, d_ijl, d_2s, s_ijl, hamiltonian, same = False):
    '''
        Random initialization of the states table (dimers) and computing 
        the initial energy
    '''
    #initialize the dimers
    states = [np.array([1 for i in range(len(d_ijl))], dtype='int32') for ignored in range(nt)]

    #initialize the spins randomly
    spinstates = [(np.random.randint(0, 2, size=len(s_ijl))*2 - 1) for i in range(nt)]
    
    #initialise the dimer state according to the spin state
    for t in range(nt):
        for id_dim in range(len(d_ijl)):
            [id_s1, id_s2] = d_2s[id_dim]
            s1 = spinstates[t][id_s1]
            s2 = spinstates[t][id_s2]
            if (s1 == s2):
                states[t][id_dim] = 1
            else:
                states[t][id_dim] = -1
    en_states = [compute_energy(hamiltonian, states[t]) for t in range(nt)] # energy computed via the function in c++
    
    return states, en_states

In [ ]:
def onestatecheck(spinstate, state, d_2s):
    '''
        This function checks whether the dimer state and the spin state are compatible
    '''
    mistakes = list()
    for id_d, d in enumerate(state):
        [id_s1, id_s2] = d_2s[id_d]
        s1 = spinstate[id_s1]
        s2 = spinstate[id_s2]
        if (s1 == s2 and d == -1):
            mistakes.append((id_d, id_s1, id_s2))
        if (s1 != s2 and d == 1):
            mistakes.append((id_d, id_s1, id_s2))
    return mistakes

In [ ]:
def statescheck(spinstates, states, d_2s):
    '''
        This function checks whether the dimer stateS and the spin stateS are compatible
    '''
    for spinstate, state in zip(spinstates, states):
        if len(onestatecheck(spinstate, state, d_2s)) != 0:
            return False
    return True


In [ ]:
def onestate_dimers2spins(sidlist, didlist, L, state):
    '''
        For a known state of the dual lattice (i.e. for each bond, is there or not a dimer), returns the corresponding
        spin state.
    '''
    #initialise the spins in a meaningless way
    spinstate = [0 for _ in range(len(sidlist))]
    #initialise the first spin randomly
    s_val =  np.random.randint(0, 2)*2-1
    s = spinstate[sidlist[0]] = s_val
    for it in range(0, len(sidlist)-1):
        db_id = didlist[it]
        spin_id = sidlist[it+1]
        s = spinstate[spin_id] = s * state[db_id]
    return spinstate

In [ ]:
def states_dimers2spins(sidlist, didlist, L, states):
    spinstates = []
    for state in states:
        spinstates.append(onestate_dimers2spins(sidlist, didlist, L, state))
    return spinstates